<a href="https://colab.research.google.com/github/emma-wall/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [74]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-07 20:50:48--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.75MB/s    in 0.2s    

2021-08-07 20:50:48 (5.75 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [75]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [76]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

### Create DataFrames


In [77]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes','vine', 'verified_purchase'])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [78]:
# Filter the Vine DataFrame/ create new DataFrame where total_votes is greater or equal to 20
votes_df = vine_df.filter(vine_df.total_votes >= 20)
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [79]:
# Filter the Votes DataFrame to show where the percentage of helpful votes is above 50%
helpful_votes_df = votes_df.withColumn("helpful_votes_percentage", votes_df.helpful_votes/votes_df.total_votes *100)
helpful_votes_df = helpful_votes_df.filter(helpful_votes_df.helpful_votes_percentage >= 50)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|       87.09677419354838|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|                   100.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|       93.54838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|       83.33333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|       96.42857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|                  96.875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|              

In [80]:
# Filter the helpful_votes_df to show review that were part of the Vine program (paid)
vine_reviews_df = helpful_votes_df.filter(helpful_votes_df.vine=="Y")
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|                    90.0|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|                   100.0|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|       92.85714285714286|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|       96.66666666666667|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|                   100.0|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|       96.15384615384616|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|              

In [81]:
# Filter the helpful_votes_df to show review that were not part of the Vine program (unpaid)
nonvine_reviews_df = helpful_votes_df.filter(helpful_votes_df.vine=="N")
nonvine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|       87.09677419354838|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|                   100.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|       93.54838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|       83.33333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|       96.42857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|                  96.875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|              

###Preform Analysis on Vine and Non-Vine Review Dataframes

In [82]:
# Find number of reviews, number of 5-star reviews and percentage of 5-star reviews 
# Vine Reviews
vine_review_count = vine_reviews_df.count()
vine_fivestar_count = vine_reviews_df.filter(vine_reviews_df.star_rating == 5).count()
vine_fivestar_percent = vine_fivestar_count/vine_review_count * 100

# None Vine Reviews
nonvine_review_count = nonvine_reviews_df.count()
nonvine_fivestar_count = nonvine_reviews_df.filter(nonvine_reviews_df.star_rating == 5).count()
nonvine_fivestar_percent = nonvine_fivestar_count/nonvine_review_count * 100

# Display in DataFrame"
columns = ['','total_reviews', 'fivestar_reviews', 'precent_fivestar_reviews']
vals = [('Vine',vine_review_count, vine_fivestar_count, vine_fivestar_percent), 
        ('Non-Vine',nonvine_review_count, nonvine_fivestar_count, nonvine_fivestar_percent)]
fivestar_df = spark.createDataFrame(vals, columns)
fivestar_df.show()

+--------+-------------+----------------+------------------------+
|        |total_reviews|fivestar_reviews|precent_fivestar_reviews|
+--------+-------------+----------------+------------------------+
|    Vine|          170|              65|       38.23529411764706|
|Non-Vine|        37840|           20612|      54.471458773784356|
+--------+-------------+----------------+------------------------+

